In [ ]:
import numpy as np
import pandas as pd
import joblib

# Load diabetes dataset
diabetes_dataset = pd.read_csv('diabetes_dataset.csv')

# Separate data and labels
X = diabetes_dataset.drop(columns='Outcome', axis=1).values
Y = diabetes_dataset['Outcome'].values

# Standardize data
def standard_scaler(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_scaled = (X - mean) / std
    return X_scaled

X = standard_scaler(X)

# Train-test split
def train_test_split(X, Y, test_size=0.2, random_state=None):
    np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    split_index = int(len(indices) * (1 - test_size))
    train_indices, test_indices = indices[:split_index], indices[split_index:]
    X_train, X_test = X[train_indices], X[test_indices]
    Y_train, Y_test = Y[train_indices], Y[test_indices]
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)
print(X.shape, X_train.shape, X_test.shape)
# SVM Classifier
class PolynomialSVM:
    def __init__(self, degree=3, learning_rate=0.01, lambda_param=0.1, n_iters=1000):
        self.degree = degree
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def _polynomial_kernel(self, x, y):
        return (1 + np.dot(x, y)) ** self.degree

    def fit(self, X, y):
        n_samples, n_features = X.shape

        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (self._polynomial_kernel(x_i, x_i) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y[idx]))
                    self.b -= self.lr * y[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

svm_classifier_scratch = PolynomialSVM(degree=3, learning_rate=0.001, lambda_param=0.01, n_iters=2000)  # Change the degree for polynomial kernel
svm_classifier_scratch.fit(X_train_pca, Y_train)

# Logistic Regression Classifier
class LogisticRegression:
    def __init__(self, learning_rate=0.001, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(linear_model)
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = self._sigmoid(linear_model)
        y_pred_class = [1 if i > 0.5 else 0 for i in y_pred]
        return np.array(y_pred_class)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

lr_classifier_scratch = LogisticRegression()
lr_classifier_scratch.fit(X_train_pca, Y_train)







 
